In [1]:
import pandas as pd
from sklearn.metrics import *
from tqdm import tqdm
from utils import Load_Rumours_Dataset_filtering_since_first_post_Transfer_Learning
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.utils.class_weight import compute_class_weight

In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
#mlflow.set_experiment("Xgboost experiments 2024-09-26 SMOTE")

In [3]:
train_dataset = 'charlie_hebdo'
#test_dataset = 'germanwings_crash'
test_dataset = 'ferguson'
time_cut =3*60*24
processor = Load_Rumours_Dataset_filtering_since_first_post_Transfer_Learning(train_dataset,\
           test_dataset, time_cut=time_cut,test_size=0.7)

processor.load_data()
processor.process_data()
train,test = processor.get_final_dataframes()

rumour
0    476
1    229
Name: count, dtype: int64


In [4]:
X_train  = train.drop(columns=['rumour'])
X_train = np.hstack([X_train.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))])
#X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
y_train =train['rumour']

X_test  = test.drop(columns=['rumour'])
X_test = np.hstack([X_test.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))])
#X = np.hstack([X.drop(columns=['embeddings_avg']).values, np.array(pd.DataFrame(X.embeddings_avg.tolist()))])
y_test =test['rumour']

In [5]:
# Compute class weights to handle imbalance
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


In [15]:
# Compute class weights to handle imbalance
classes = np.unique(y_train)
class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))

# Initialize Random Forest with class weights
model = RandomForestClassifier(
    n_estimators=250,
    max_depth=2,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features='sqrt',
    class_weight=class_weight_dict,  # Handles imbalance
    n_jobs=-1,
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

# Predict and evaluate

y_proba_test = model.predict_proba(X_test)[:, 1]  # For ROC AUC

y_proba_train = model.predict_proba(X_train)[:, 1]  # For ROC AUC

In [16]:

thresholds = np.linspace(0.01, 0.99, 100)
f1_scores = [f1_score(y_train, (y_proba_train > t).astype(int)) for t in thresholds]
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[best_idx]

y_train_pred = (y_proba_train > best_threshold).astype(int)
y_test_pred = (y_proba_test > best_threshold).astype(int)

# Evaluation function
def evaluate(y_true, y_pred, y_prob, label=""):
    print(f"  - Accuracy:  {accuracy_score(y_true, y_pred):.4f}")
    print(f"  - Precision: {precision_score(y_true, y_pred):.4f}")
    print(f"  - Recall:    {recall_score(y_true, y_pred):.4f}")
    print(f"  - AUC:       {roc_auc_score(y_true, y_prob):.4f}")
    print("")

# Show metrics
evaluate(y_train, y_train_pred, y_proba_train, label="Train")
evaluate(y_test, y_test_pred, y_proba_test, label="Test")

  - Accuracy:  0.8560
  - Precision: 0.6265
  - Recall:    0.7453
  - AUC:       0.8911

  - Accuracy:  0.7191
  - Precision: 0.7925
  - Recall:    0.1834
  - AUC:       0.7306



In [3]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
#mlflow.set_experiment("spyder-experiment")
import mlflow.pytorch
mlflow.set_experiment("Random Forest  2025-06-22 Sydney Siege TF")

2025/06/22 18:01:56 INFO mlflow.tracking.fluent: Experiment with name 'Random Forest  2025-06-22 Sydney Siege TF' does not exist. Creating a new experiment.


<Experiment: artifact_location='/workspaces/rumour-detection-gnn/mlruns/90', creation_time=1750615316896, experiment_id='90', last_update_time=1750615316896, lifecycle_stage='active', name='Random Forest  2025-06-22 Sydney Siege TF', tags={}>

#### Testing draft model

In [4]:
for time_cut in range(10, (60 * 24 * 3), 15):
    print(f"\n=== Time Cut: {time_cut} ===")
    
    train_dataset = 'charlie_hebdo'
    #test_dataset = 'germanwings_crash'
    #test_dataset = 'ottawashooting'
    test_dataset = 'sydneysiege'
    #test_dataset = 'ferguson'
    time_cut =time_cut
    processor = Load_Rumours_Dataset_filtering_since_first_post_Transfer_Learning(train_dataset,\
               test_dataset, time_cut=time_cut,test_size=0.7)
    
    processor.load_data()
    processor.process_data()
    train,test = processor.get_final_dataframes()

    X_train = train.drop(columns=['rumour'])
    X_train = np.hstack([
        X_train.drop(columns=['embeddings_avg']).values, 
        np.array(pd.DataFrame(X_train.embeddings_avg.tolist()))
    ])
    y_train = train['rumour']

    X_test = test.drop(columns=['rumour'])
    X_test = np.hstack([
        X_test.drop(columns=['embeddings_avg']).values, 
        np.array(pd.DataFrame(X_test.embeddings_avg.tolist()))
    ])
    y_test = test['rumour']

    # Compute class weights to handle imbalance
    classes = np.unique(y_train)
    class_weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)
    class_weight_dict = dict(zip(classes, class_weights))
    
    model = RandomForestClassifier(
        n_estimators=50,
        max_depth=2,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features='sqrt',
        class_weight=class_weight_dict,
        n_jobs=-1,
        random_state=42
    )

    with mlflow.start_run(run_name=f"time_cut_{time_cut}"):
        model.fit(X_train, y_train)

        # Probabilities
        y_train_prob = model.predict_proba(X_train)[:, 1]
        y_test_prob = model.predict_proba(X_test)[:, 1]

        # Find best threshold to maximize F1 on train set
        #thresholds = np.linspace(0.01, 0.99, 100)
        #f1_scores = [f1_score(y_train, (y_train_prob > t).astype(int)) for t in thresholds]
        #best_idx = np.argmax(f1_scores)
        #best_threshold = thresholds[best_idx]
        best_threshold = 0.5

        # Apply best threshold
        y_train_pred = (y_train_prob > best_threshold).astype(int)
        y_test_pred = (y_test_prob > best_threshold).astype(int)

        # Train metrics
        mlflow.log_metric("train_accuracy", accuracy_score(y_train, y_train_pred))
        mlflow.log_metric("train_precision", precision_score(y_train, y_train_pred))
        mlflow.log_metric("train_recall", recall_score(y_train, y_train_pred))
        mlflow.log_metric("train_f1", f1_score(y_train, y_train_pred))
        mlflow.log_metric("train_auc", roc_auc_score(y_train, y_train_prob))

        # Test metrics
        mlflow.log_metric("final_acc", accuracy_score(y_test, y_test_pred))
        mlflow.log_metric("final_precision", precision_score(y_test, y_test_pred))
        mlflow.log_metric("final_recall", recall_score(y_test, y_test_pred))
        mlflow.log_metric("final_f1", f1_score(y_test, y_test_pred))
        mlflow.log_metric("final_auc", roc_auc_score(y_test, y_test_prob))

        # Log best threshold and time_cut
        mlflow.log_metric("optimal_threshold", best_threshold)
        mlflow.log_metric("time_cut", time_cut)



=== Time Cut: 10 ===
rumour
0    5
1    2
Name: count, dtype: int64

=== Time Cut: 25 ===
rumour
0    15
1     7
Name: count, dtype: int64

=== Time Cut: 40 ===
rumour
0    26
1    16
Name: count, dtype: int64

=== Time Cut: 55 ===
rumour
0    39
1    21
Name: count, dtype: int64

=== Time Cut: 70 ===
rumour
0    47
1    25
Name: count, dtype: int64

=== Time Cut: 85 ===
rumour
0    56
1    26
Name: count, dtype: int64

=== Time Cut: 100 ===
rumour
0    61
1    34
Name: count, dtype: int64

=== Time Cut: 115 ===
rumour
0    65
1    46
Name: count, dtype: int64

=== Time Cut: 130 ===
rumour
0    69
1    59
Name: count, dtype: int64

=== Time Cut: 145 ===
rumour
0    73
1    66
Name: count, dtype: int64

=== Time Cut: 160 ===
rumour
0    81
1    69
Name: count, dtype: int64

=== Time Cut: 175 ===
rumour
0    88
1    74
Name: count, dtype: int64

=== Time Cut: 190 ===
rumour
0    92
1    86
Name: count, dtype: int64

=== Time Cut: 205 ===
rumour
1    99
0    97
Name: count, dtype: int64
